In [4]:
import torch
import torchvision
import numpy as np
import sys
import d2l.torch as d2l
import torch.nn as nn

In [11]:
from torch.nn import init
batch_size = 256
train_iter,test_iter = d2l.load_data_fashion_mnist(batch_size)

num_inputs = 784
num_outputs = 10
# 初始化模型参数 w 和 b
w = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)
class LinearNet(nn.Module):
    def __init__(self,num_inputs,num_outputs):
        super(LinearNet,self).__init__()
        self.linear = nn.Linear(num_inputs,num_outputs)
    def forward(self,x):
        y = self.linear(x.view(x.shape[0],-1))
        return y
net = LinearNet(num_inputs,num_outputs)

class FlattenLayer(nn.Module):
    def __init__(self):
        super(FlattenLayer, self).__init__()
    
    def forward(self, x):  # x shape: (batch, *, *, ...)
        return x.view(x.shape[0], -1)
from collections import OrderedDict
net = nn.Sequential(
 # FlattenLayer(),
 # nn.Linear(num_inputs, num_outputs)
 OrderedDict([
 ('flatten', FlattenLayer()),
 ('linear', nn.Linear(num_inputs, num_outputs))])
 )
init.normal_(net.linear.weight, mean=0, std=0.01)
init.constant_(net.linear.bias, val=0)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

定义优化算法

In [7]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

训练模型

In [13]:
loss = nn.CrossEntropyLoss()
# 辅助函数
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
            n += y.shape[0]
    return acc_sum / n

num_epochs,lr = 5, 0.1
def train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,params=None,lr=None,optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X,y in train_iter:
            y_hat = net(X)
            l = loss(y_hat,y).sum()
            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            if optimizer is None:
                sgd(params,lr,batch_size)
            else:
                optimizer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'% (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))
# 修复后的SGD函数：增加对grad是否为None的判断
def sgd(params, lr, batch_size):
    """小批量随机梯度下降算法"""
    if lr is None or batch_size is None:
        raise ValueError("学习率(lr)和批量大小(batch_size)不能为None")
    with torch.no_grad():
        for param in params:
            if param.grad is not None:
                param -= lr * param.grad / batch_size

# 确保以下变量已正确定义：
# net, train_iter, test_iter, w, b, batch_size

# 调用训练函数
train_ch3(net, train_iter, test_iter, loss, num_epochs, batch_size, [w, b], lr)

epoch 1, loss 0.0091, train acc 0.136, test acc 0.136
epoch 2, loss 0.0091, train acc 0.136, test acc 0.136
epoch 3, loss 0.0091, train acc 0.136, test acc 0.136
epoch 4, loss 0.0091, train acc 0.136, test acc 0.136
epoch 5, loss 0.0091, train acc 0.136, test acc 0.136
